<a href="https://colab.research.google.com/github/QiaoYe-Learning/NLP-classwork/blob/main/Assignment_1_Simple_Sentiment_Analysis_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# download dataset
!pip install torchtext==0.4.0
# import
import torch
from torchtext import data
from torchtext import datasets # pre-define dataset supported by torchtext
import random
import torch.nn as nn # Neural Network
import time
import torch.optim as optim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.5 MB/s eta 0:00:00


In [2]:
# Model Building And Function Defining
class RNN(nn.Module): # inherited from nn.Module
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim): # vocabulary size, embedding_dim, hidden_dim, classification number

        super().__init__()

        self.embedding = nn.Embedding(input_dim, embedding_dim)

        self.rnn = nn.RNN(embedding_dim, hidden_dim)

        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text, text_lengths):
        #text = [sentence length, batch size], text_lengths: real length

        embedded = self.embedding(text)
        #embedded = [sent len, batch size, emb dim]

        output, hidden = self.rnn(embedded)
        #output = [sent len, batch size, hid dim] - The hidden status of each time step, containing information about all time steps.
        #hidden = [1, batch size, hid dim] - The hidden state of the last time step

        #assert torch.equal(output[-1,:,:], hidden.squeeze(0)) - Verify that the output of the last time step is equal to the hidden state

        return self.fc(hidden.squeeze(0))

def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division
    acc = correct.sum() / len(correct)
    return acc

def train(model, iterator, optimizer, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.train() # set model - train

    for batch in iterator:

        optimizer.zero_grad() # Clear the gradient in the optimizer before each forward propagation

        text, text_lengths = batch.text

        predictions = model(text, text_lengths).squeeze(1)

        loss = criterion(predictions, batch.label)

        acc = binary_accuracy(predictions, batch.label)

        loss.backward() # Calculate the gradient of the loss by back propagation

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.eval() # setting model - evaluate

    with torch.no_grad(): # No back propagation is required during the evaluation process, so the gradient calculation is turned off

        for batch in iterator:
            text, text_lengths = batch.text

            predictions = model(text, text_lengths).squeeze(1)

            loss = criterion(predictions, batch.label)

            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [3]:
# Initial setting
SEED = 1234 # random seed
torch.manual_seed(SEED) # normalize random seed

torch.backends.cudnn.deterministic = True # enture CUDA's behavior is stable（determinacy）

TEXT = data.Field(tokenize='spacy', # split word using spacy
                  tokenizer_language = 'en_core_web_sm', # enture spacy using English_model
                  include_lengths = True, # include the length of sentence when return the data
                  pad_first=True) # add a mark before the sequence
LABEL = data.LabelField(dtype = torch.float) # specfies the tyoe of label data is 'float'(binary classfication and regression)

training_data, test_data = datasets.IMDB.splits(TEXT, LABEL) # load dataset-IMDB(binary classfication task)
train_data, valid_data = training_data.split(random_state = random.seed(SEED)) # Default 70/30
# train_data, valid_data = train_data.split(split_ratio=0.8, random_state=random.seed(SEED)) 80/20 = train/valid


# Build a vocabulary
MAX_VOCAB_SIZE = 25_000 # max size of vocabulary, only top 25000, other is <unk>
TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE) # build train vocabulary
LABEL.build_vocab(train_data) # build vicabulary for the label fields <unk>,<pad>


# Using BucketIterator to create data iterator for train, valid, test to processes data in batch
BATCH_SIZE = 64 # size of samples per batch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # device choice, preferential using GPU(cuda)
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    sort_within_batch = True, # sentences in each batch are sorted by their length, useful to reduce the using of <pad>
    device = device)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:03<00:00, 26.2MB/s]


In [10]:
# Model Settings
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
model_RNN_main = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [11]:
# Main-RNN
N_EPOCHS = 20
criterion = nn.BCEWithLogitsLoss() # Loss function of binary task
model_RNN = model_RNN_main.to(device)
criterion = criterion.to(device)

# Optimizer Settings
optimizer_SGD = optim.SGD(model_RNN_main.parameters(), lr=1e-3) # SGD optimizer - Stochastic gradient descent
optimizer_Adam = optim.Adam(model_RNN_main.parameters(), lr=1e-3) # Adam optimizer - Stochastic gradient descent
optimizer_Adagrad = optim.Adagrad(model_RNN_main.parameters(), lr=1e-3) # Adagrad optimizer - Stochastic gradient descent

In [7]:
## SGD
best_valid_loss_SGD_20 = float('inf')
epoch=0
for epoch in range(N_EPOCHS):
    start_time_SGD_20 = time.time()

    train_loss_SGD_20, train_acc_SGD_20 = train(model_RNN, train_iterator, optimizer_SGD, criterion)
    valid_loss_SGD_20, valid_acc_SGD_20 = evaluate(model_RNN, valid_iterator, criterion)

    end_time_SGD_20 = time.time()

    epoch_mins_SGD_20, epoch_secs_SGD_20 = epoch_time(start_time_SGD_20, end_time_SGD_20)

    if valid_loss_SGD_20 < best_valid_loss_SGD_20:
        best_valid_loss = valid_loss_SGD_20
        torch.save(model_RNN.state_dict(), 'tut1-model_SGD_20.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins_SGD_20}m {epoch_secs_SGD_20}s')
    print(f'\tTrain Loss-SGD_20: {train_loss_SGD_20:.3f} | Train Acc-SGD_20: {train_acc_SGD_20*100:.2f}%')
    print(f'\t Val. Loss-SGD_20: {valid_loss_SGD_20:.3f} |  Val. Acc-SGD_20: {valid_acc_SGD_20*100:.2f}%')

# SGD_20
model_RNN.load_state_dict(torch.load('tut1-model_SGD_20.pt'))
test_loss_SGD_20, test_acc_SGD_20 = evaluate(model_RNN, test_iterator, criterion)
print(f'Test Loss_SGD_20: {test_loss_SGD_20:.3f} | Test Acc_SGD_20: {test_acc_SGD_20*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 5s
	Train Loss-SGD_20: 0.693 | Train Acc-SGD_20: 51.93%
	 Val. Loss-SGD_20: 0.693 |  Val. Acc-SGD_20: 51.86%
Epoch: 02 | Epoch Time: 0m 4s
	Train Loss-SGD_20: 0.691 | Train Acc-SGD_20: 52.26%
	 Val. Loss-SGD_20: 0.692 |  Val. Acc-SGD_20: 51.81%
Epoch: 03 | Epoch Time: 0m 3s
	Train Loss-SGD_20: 0.690 | Train Acc-SGD_20: 52.63%
	 Val. Loss-SGD_20: 0.692 |  Val. Acc-SGD_20: 51.55%
Epoch: 04 | Epoch Time: 0m 3s
	Train Loss-SGD_20: 0.690 | Train Acc-SGD_20: 52.70%
	 Val. Loss-SGD_20: 0.691 |  Val. Acc-SGD_20: 51.88%
Epoch: 05 | Epoch Time: 0m 4s
	Train Loss-SGD_20: 0.689 | Train Acc-SGD_20: 52.92%
	 Val. Loss-SGD_20: 0.690 |  Val. Acc-SGD_20: 52.41%
Epoch: 06 | Epoch Time: 0m 3s
	Train Loss-SGD_20: 0.688 | Train Acc-SGD_20: 53.15%
	 Val. Loss-SGD_20: 0.690 |  Val. Acc-SGD_20: 52.51%
Epoch: 07 | Epoch Time: 0m 3s
	Train Loss-SGD_20: 0.688 | Train Acc-SGD_20: 53.41%
	 Val. Loss-SGD_20: 0.689 |  Val. Acc-SGD_20: 53.24%
Epoch: 08 | Epoch Time: 0m 4s
	Train Loss-SGD_20

<ipython-input-7-8b315b528752>:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_RNN.load_state_dict(torch.load('tut1-model_SGD_20.pt'))


Test Loss_SGD_20: 0.684 | Test Acc_SGD_20: 55.45%


In [8]:
## Adam_20
best_valid_loss_Adam_20 = float('inf')
epoch=0
for epoch in range(N_EPOCHS):
    start_time_Adam_20 = time.time()

    train_loss_Adam_20, train_acc_Adam_20 = train(model_RNN, train_iterator, optimizer_Adam, criterion)
    valid_loss_Adam_20, valid_acc_Adam_20 = evaluate(model_RNN, valid_iterator, criterion)

    end_time_Adam_20 = time.time()

    epoch_mins_Adam_20, epoch_secs_Adam_20 = epoch_time(start_time_Adam_20, end_time_Adam_20)

    if valid_loss_Adam_20 < best_valid_loss_Adam_20:
        best_valid_loss = valid_loss_Adam_20
        torch.save(model_RNN.state_dict(), 'tut1-model_Adam_20.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins_Adam_20}m {epoch_secs_Adam_20}s')
    print(f'\tTrain Loss-Adam_20: {train_loss_Adam_20:.3f} | Train Acc-Adam_20: {train_acc_Adam_20*100:.2f}%')
    print(f'\t Val. Loss-Adam_20: {valid_loss_Adam_20:.3f} |  Val. Acc-Adam_20: {valid_acc_Adam_20*100:.2f}%')

# Adam_20
model_RNN.load_state_dict(torch.load('tut1-model_Adam_20.pt'))
test_loss_Adam_20, test_acc_Adam_20 = evaluate(model_RNN, test_iterator, criterion)
print(f'Test Loss_Adam_20: {test_loss_Adam_20:.3f} | Test Acc_Adam_20: {test_acc_Adam_20*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 4s
	Train Loss-Adam_20: 0.677 | Train Acc-Adam_20: 57.53%
	 Val. Loss-Adam_20: 0.710 |  Val. Acc-Adam_20: 48.16%
Epoch: 02 | Epoch Time: 0m 4s
	Train Loss-Adam_20: 0.644 | Train Acc-Adam_20: 62.35%
	 Val. Loss-Adam_20: 0.614 |  Val. Acc-Adam_20: 67.10%
Epoch: 03 | Epoch Time: 0m 4s
	Train Loss-Adam_20: 0.575 | Train Acc-Adam_20: 69.85%
	 Val. Loss-Adam_20: 0.579 |  Val. Acc-Adam_20: 71.28%
Epoch: 04 | Epoch Time: 0m 3s
	Train Loss-Adam_20: 0.536 | Train Acc-Adam_20: 73.79%
	 Val. Loss-Adam_20: 0.575 |  Val. Acc-Adam_20: 72.33%
Epoch: 05 | Epoch Time: 0m 3s
	Train Loss-Adam_20: 0.479 | Train Acc-Adam_20: 77.79%
	 Val. Loss-Adam_20: 0.576 |  Val. Acc-Adam_20: 71.56%
Epoch: 06 | Epoch Time: 0m 4s
	Train Loss-Adam_20: 0.546 | Train Acc-Adam_20: 71.43%
	 Val. Loss-Adam_20: 0.667 |  Val. Acc-Adam_20: 60.40%
Epoch: 07 | Epoch Time: 0m 3s
	Train Loss-Adam_20: 0.656 | Train Acc-Adam_20: 58.66%
	 Val. Loss-Adam_20: 0.693 |  Val. Acc-Adam_20: 51.43%
Epoch: 08 | Epoch Ti

<ipython-input-8-42a6fdf30cbd>:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_RNN.load_state_dict(torch.load('tut1-model_Adam_20.pt'))


Test Loss_Adam_20: 0.693 | Test Acc_Adam_20: 73.51%


In [12]:
## Adagrad_20
best_valid_loss_Adagrad_20 = float('inf')
epoch = 0
for epoch in range(N_EPOCHS):
    start_time_Adagrad_20 = time.time()

    train_loss_Adagrad_20, train_acc_Adagrad_20 = train(model_RNN, train_iterator, optimizer_Adagrad, criterion)
    valid_loss_Adagrad_20, valid_acc_Adagrad_20 = evaluate(model_RNN, valid_iterator, criterion)

    end_time_Adagrad_20 = time.time()

    epoch_mins_Adagrad_20, epoch_secs_Adagrad_20 = epoch_time(start_time_Adagrad_20, end_time_Adagrad_20)

    if valid_loss_Adagrad_20 < best_valid_loss_Adagrad_20:
        best_valid_loss_Adagrad_20 = valid_loss_Adagrad_20
        torch.save(model_RNN.state_dict(), 'tut1-model_Adagrad_20.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins_Adagrad_20}m {epoch_secs_Adagrad_20}s')
    print(f'\tTrain Loss-Adagrad_20: {train_loss_Adagrad_20:.3f} | Train Acc-Adagrad_20: {train_acc_Adagrad_20*100:.2f}%')
    print(f'\t Val. Loss-Adagrad_20: {valid_loss_Adagrad_20:.3f} |  Val. Acc-Adagrad_20: {valid_acc_Adagrad_20*100:.2f}%')

# Adagrad_20
model_RNN.load_state_dict(torch.load('tut1-model_Adagrad_20.pt'))
test_loss_Adagrad_20, test_acc_Adagrad_20 = evaluate(model_RNN, test_iterator, criterion)
print(f'Test Loss_Adagrad_20: {test_loss_Adagrad_20:.3f} | Test Acc_Adagrad_20: {test_acc_Adagrad_20*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 5s
	Train Loss-Adagrad_20: 0.683 | Train Acc-Adagrad_20: 55.33%
	 Val. Loss-Adagrad_20: 0.675 |  Val. Acc-Adagrad_20: 57.30%
Epoch: 02 | Epoch Time: 0m 3s
	Train Loss-Adagrad_20: 0.645 | Train Acc-Adagrad_20: 62.39%
	 Val. Loss-Adagrad_20: 0.638 |  Val. Acc-Adagrad_20: 63.41%
Epoch: 03 | Epoch Time: 0m 3s
	Train Loss-Adagrad_20: 0.641 | Train Acc-Adagrad_20: 63.44%
	 Val. Loss-Adagrad_20: 0.665 |  Val. Acc-Adagrad_20: 59.12%
Epoch: 04 | Epoch Time: 0m 4s
	Train Loss-Adagrad_20: 0.642 | Train Acc-Adagrad_20: 63.58%
	 Val. Loss-Adagrad_20: 0.651 |  Val. Acc-Adagrad_20: 61.70%
Epoch: 05 | Epoch Time: 0m 3s
	Train Loss-Adagrad_20: 0.629 | Train Acc-Adagrad_20: 64.42%
	 Val. Loss-Adagrad_20: 0.638 |  Val. Acc-Adagrad_20: 63.40%
Epoch: 06 | Epoch Time: 0m 3s
	Train Loss-Adagrad_20: 0.619 | Train Acc-Adagrad_20: 65.57%
	 Val. Loss-Adagrad_20: 0.631 |  Val. Acc-Adagrad_20: 64.43%
Epoch: 07 | Epoch Time: 0m 4s
	Train Loss-Adagrad_20: 0.611 | Train Acc-Adagrad_20: 66.3

<ipython-input-12-9acf827037f4>:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_RNN.load_state_dict(torch.load('tut1-model_Adagrad_20.pt'))


Test Loss_Adagrad_20: 0.596 | Test Acc_Adagrad_20: 68.62%
